# 00 Quickstart: DINOv3 Features

This notebook demonstrates how to load the DINOv3 backbone and visualize features using PCA.

In [ ]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
from transformers import AutoImageProcessor
from sklearn.decomposition import PCA

from dinov3_lab.core.backbone import build_dinov3_hf

# 1. Load Model
model_id = "facebook/dinov3-vitl16-pretrain-lvd1689m"
backbone = build_dinov3_hf(model_id=model_id)
processor = AutoImageProcessor.from_pretrained(model_id)

# 2. Load Image
# Replace with your image path
image_path = "../data/test_images/demo.jpg" 
# !wget https://github.com/pytorch/hub/raw/master/images/dog.jpg -O ../data/test_images/demo.jpg
try:
    image = Image.open(image_path).convert("RGB")
except FileNotFoundError:
    print("Please download a demo image to ../data/test_images/demo.jpg")
    # Create a dummy image
    image = Image.new('RGB', (448, 448), color = 'red')

inputs = processor(images=image, return_tensors="pt")

# 3. Extract Features
with torch.no_grad():
    out = backbone(inputs.pixel_values)

print(f"CLS shape: {out.cls.shape}")
print(f"Patch tokens shape: {out.patch_tokens.shape}")

# 4. Visualize PCA
patch_tokens = out.patch_tokens[0].float().cpu().numpy()
pca = PCA(n_components=3)
pca_features = pca.fit_transform(patch_tokens)

h, w = out.patch_hw
pca_img = pca_features.reshape(h, w, 3)

# Normalize to 0-1 for display
pca_img = (pca_img - pca_img.min()) / (pca_img.max() - pca_img.min())

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(pca_img)
plt.title("PCA of Features")
plt.axis("off")
plt.show()